In [1]:
#NAVI_HARD_CODE_DOMAIN

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os
import pandas as pd
#Functional coding
import functools
from functools import partial
from tensorflow.python.ops import array_ops 

In [3]:
#Data Path..
Datapath="DATA/Navigation/Navigation_Data.txt"
Labelpath="DATA/Navigation/Navigation_Label.txt"
Rewardpath="DATA/Navigation/Navigation_Reward.txt"

In [4]:
#Given local path, find full path
def PathFinder(path):
    #python 2
    #script_dir = os.path.dirname('__file__')
    #fullpath = os.path.join(script_dir,path)
    #python 3
    fullpath=os.path.abspath(path)
    print(fullpath)
    return fullpath

#Read Data for Deep Learning
def ReadData(path):
    fullpath=PathFinder(path)
    return pd.read_csv(fullpath, sep=',', header=0)

In [5]:
S_A_pd = ReadData(Datapath)
SP_pd = ReadData(Labelpath)
R_pd = ReadData(Rewardpath)
S_A_matrix=S_A_pd.as_matrix()
SP_matrix=SP_pd.as_matrix()
R_matrix=R_pd.as_matrix()

/home/wuga/Documents/Notebook/VAE-PLANNING/DATA/Navigation/Navigation_Data.txt
/home/wuga/Documents/Notebook/VAE-PLANNING/DATA/Navigation/Navigation_Label.txt
/home/wuga/Documents/Notebook/VAE-PLANNING/DATA/Navigation/Navigation_Reward.txt


In [6]:
default_settings = {
    "dims"          : 2,
    "min_maze_bound": tf.constant(0.0,dtype=tf.float32), 
    "max_maze_bound": tf.constant(10.0,dtype=tf.float32), 
    "min_act_bound": tf.constant(-1.0,dtype=tf.float32), 
    "max_act_bound": tf.constant(1.0,dtype=tf.float32), 
    "goal"    : tf.constant(8.0,dtype=tf.float32),
    "penalty" : tf.constant(1000000.0,dtype=tf.float32),
    "centre"  : tf.constant(5.0,dtype=tf.float32)
   }

In [7]:
class NAVI(object):
    def __init__(self, 
                 default_settings):
        self.__dict__.update(default_settings)
        self.zero = tf.constant(0,dtype=tf.float32)
        self.two = tf.constant(2.0,dtype=tf.float32)
        self.one = tf.constant(1.0,dtype=tf.float32)
        self.lessone = tf.constant(0.99,dtype=tf.float32)
    
    def MINMAZEBOUND(self, dim):
        return self.min_maze_bound
    
    def MAXMAZEBOUND(self, dim):
        return self.max_maze_bound
    
    def MINACTIONBOUND(self, dim):
        return self.min_act_bound
    
    def MAXACTIONBOUND(self, dim):
        return self.max_act_bound
    
    def GOAL(self, dim):
        return self.goal
    
    def CENTER(self, dim):
        return self.centre
    
    def PENALTY(self):
        return self.penalty
    
    def _transition(self, dim, states_packed, actions_packed):
        
        states = tf.unpack(states_packed)
        actions = tf.unpack(actions_packed)
        
        previous_state = states[dim]
        
        #distance to centre Manhattan
        distance = self.zero
        for i in range(len(states)):
            distance+=tf.abs(states[i]-self.CENTER(i))
        
        #scale factor
        scalefactor = tf.cond(distance<self.two, lambda: distance/self.two, lambda: self.one)
        
        #proposed location
        proposedLoc = previous_state + actions[dim]*scalefactor
        
        #new state
        new_state = tf.cond(tf.logical_and(proposedLoc <= self.MAXMAZEBOUND(dim), proposedLoc >= self.MINMAZEBOUND(dim)), \
                            lambda: proposedLoc, \
                            lambda: tf.cond(proposedLoc >self.MAXMAZEBOUND(dim), lambda:self.MAXMAZEBOUND(dim), lambda:self.MINMAZEBOUND(dim) ) \
                           )
        
        return new_state
    
    # For single data point
    def _vector_trans(self, state_size, states_packed, actions_packed):
        new_states=[]
        for i in range(state_size):
            new_states.append(self._transition(i,states_packed,actions_packed))
        return tf.pack(new_states)
    
    def Transition(self, states, actions):
        new_states = []
        batch_size,state_size = states.get_shape()
        states_list = tf.unpack(states)
        actions_list = tf.unpack(actions)
        for i in range(batch_size):
            new_states.append(self._vector_trans(state_size,states_list[i],actions_list[i]))
        return tf.pack(new_states)
    
    def _reward(self, state_size, states_packed, actions_packed):
        reward = self.zero
        states = tf.unpack(states_packed)
        actions = tf.unpack(actions_packed)
        
        for i in range(state_size):
            reward -= tf.abs(states[i]-self.GOAL(i))
        return tf.pack([reward])
    
    def Reward(self, states,actions):
        new_rewards = []
        batch_size,state_size = states.get_shape()
        states_list = tf.unpack(states)
        actions_list = tf.unpack(actions)
        for i in range(batch_size):
            new_rewards.append(self._reward(state_size,states_list[i],actions_list[i]))
        return tf.pack(new_rewards)

In [8]:
# States
states = tf.placeholder(tf.float32,[10, 2],name="States")

# Actions
actions = tf.placeholder(tf.float32,[10, 2],name="Actions")


In [9]:
navi_inst = NAVI(default_settings)

In [10]:
states_list=tf.unpack(states)
actions_list = tf.unpack(actions)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
feed_dict={states:S_A_matrix[:10,2:], actions:S_A_matrix[:10,:2]}
new_state = navi_inst._transition(1,states_list[0],actions_list[0])
print(sess.run([new_state], feed_dict=feed_dict))
print(sess.run([states_list[1]], feed_dict=feed_dict))
print(sess.run([states_list[1]], feed_dict=feed_dict))

[0.0]
[array([ 0.5490576,  0.       ], dtype=float32)]
[array([ 0.5490576,  0.       ], dtype=float32)]


In [11]:
new_rewards = navi_inst.Reward(states,actions)

In [12]:
feed_dict={states:S_A_matrix[:10,2:], actions:S_A_matrix[:10,:2]}
sess.run(new_rewards,feed_dict=feed_dict )

array([[-16.        ],
       [-15.45094299],
       [-15.34755898],
       [-14.92229462],
       [-13.87710953],
       [-12.88495827],
       [-11.48205948],
       [-13.36318207],
       [-13.89812565],
       [-12.66272354]], dtype=float32)

In [13]:
class NAVICell(tf.nn.rnn_cell.RNNCell):

    def __init__(self, default_settings):
        self._num_state_units = 2
        self._num_reward_units = 3
        self.navi = NAVI(default_settings)

    @property
    def state_size(self):
        return self._num_state_units

    @property
    def output_size(self):
        return self._num_reward_units

    def __call__(self, inputs, state, scope=None):
        next_state =  self.navi.Transition(state, inputs)
        reward = self.navi.Reward(state, inputs)      
        return tf.concat(1,[reward,next_state]), next_state

In [14]:
class ActionOptimizer(object):
    def __init__(self,
                a, # Actions
                num_step, # Number of RNN step, this is a fixed step RNN sequence, 12 for navigation
                num_act, # Number of actions
                batch_size, #Batch Size
                learning_rate=0.01): 
        self.action = tf.reshape(a,[-1,num_step,num_act]) #Reshape rewards
        print(self.action)
        self.batch_size = batch_size
        self.num_step = num_step
        self.learning_rate = learning_rate
        self._p_create_rnn_graph()
        self._p_create_loss()
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())
    
    def _p_create_rnn_graph(self):
        cell = NAVICell(default_settings)
        initial_state = cell.zero_state(self.batch_size, dtype=tf.float32)
        print('action batch size:{0}'.format(array_ops.shape(self.action)[0]))
        print('Initial_state shape:{0}'.format(initial_state))
        rnn_outputs, state = tf.nn.dynamic_rnn(cell, self.action, dtype=tf.float32,initial_state=initial_state)
        #need output intermediate states as well
        concated = tf.concat(0,rnn_outputs)
        print('concated shape:{0}'.format(concated.get_shape()))
        something_unpacked =  tf.unpack(concated, axis=2)
        self.outputs = tf.reshape(something_unpacked[0],[-1,self.num_step,1])
        print(' self.outputs:{0}'.format(self.outputs.get_shape()))
        self.intern_states = tf.pack([something_unpacked[1],something_unpacked[2]], axis=2)
        self.last_state = state
        self.pred = tf.reduce_sum(self.outputs,1)
        print("self.pred:{0}".format(self.pred))
            
    def _p_create_loss(self):

        objective = tf.reduce_mean(self.pred) 
        self.loss = -objective
        print(self.loss.get_shape())
        #self.loss = -objective
        self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss, var_list=[a])
        
    def Optimize(self,epoch=100):
        
        new_loss = self.sess.run([self.loss])
        print('Loss in epoch {0}: {1}'.format("Initial", new_loss)) 
        for epoch in range(epoch):
            training = self.sess.run([self.optimizer])
            self.sess.run(tf.assign(a, tf.clip_by_value(a, -1, 1)))
            if True:
                new_loss = self.sess.run([self.loss])
                print('Loss in epoch {0}: {1}'.format(epoch, new_loss))  
        minimum_costs_id=self.sess.run(tf.argmax(self.pred,0))
        print(minimum_costs_id)
        best_action = np.round(self.sess.run(self.action)[minimum_costs_id[0]],4)
        print('Optimal Action Squence:{0}'.format(best_action))
        print('Best Cost: {0}'.format(self.sess.run(self.pred)[minimum_costs_id[0]]))
        print('The last state:{0}'.format(self.sess.run(self.last_state)[minimum_costs_id[0]]))
        print('Rewards each time step:{0}'.format(self.sess.run(self.outputs)[minimum_costs_id[0]]))
        print('Intermediate states:{0}'.format(self.sess.run(self.intern_states)[minimum_costs_id[0]]))
        

In [15]:
a = tf.Variable(tf.truncated_normal(shape=[240],mean=0.0, stddev=0.5),name="action")
rnn_inst = ActionOptimizer(a, 12,2,10)  

Tensor("Reshape:0", shape=(10, 12, 2), dtype=float32)
action batch size:Tensor("strided_slice:0", shape=(), dtype=int32)
Initial_state shape:Tensor("zeros:0", shape=(10, 2), dtype=float32)
concated shape:(10, 12, 3)
 self.outputs:(10, 12, 1)
self.pred:Tensor("Sum:0", shape=(10, 1), dtype=float32)
()


In [16]:
rnn_inst.Optimize(500)

Loss in epoch Initial: [181.99092]
Loss in epoch 0: [181.71375]
Loss in epoch 1: [181.42714]
Loss in epoch 2: [181.1235]
Loss in epoch 3: [180.80769]
Loss in epoch 4: [180.48552]
Loss in epoch 5: [180.14731]
Loss in epoch 6: [179.77074]
Loss in epoch 7: [179.38864]
Loss in epoch 8: [179.00099]
Loss in epoch 9: [178.60785]
Loss in epoch 10: [178.20905]
Loss in epoch 11: [177.8093]
Loss in epoch 12: [177.39758]
Loss in epoch 13: [176.9805]
Loss in epoch 14: [176.55812]
Loss in epoch 15: [176.13034]
Loss in epoch 16: [175.69635]
Loss in epoch 17: [175.25204]
Loss in epoch 18: [174.80276]
Loss in epoch 19: [174.35126]
Loss in epoch 20: [173.89351]
Loss in epoch 21: [173.41452]
Loss in epoch 22: [172.9268]
Loss in epoch 23: [172.44106]
Loss in epoch 24: [171.93538]
Loss in epoch 25: [171.38838]
Loss in epoch 26: [170.84492]
Loss in epoch 27: [170.30089]
Loss in epoch 28: [169.75261]
Loss in epoch 29: [169.1906]
Loss in epoch 30: [168.62761]
Loss in epoch 31: [168.06905]
Loss in epoch 32: [1